In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

from matplotlib import pyplot as plt
from dotenv import load_dotenv

from models.linear_regressions import Linear_reg

load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100


In [ ]:
file = file_ghg_eui = "./data/seattle_predict_ghg_eui.csv"

df_model = pd.read_csv(file_ghg_eui).astype(float)

df_model.columns


In [ ]:
# Target_1 : target_GHGEmissionsIntensity(kgCO2e/ft2) : two cols

droplist_1 = [
    "scaled_GHGEmissionsIntensity(kgCO2e/ft2)",  # Scaled target
    "target_SourceEUI(kWh/m2)"  # not to scale
    ]

df_model_ghg = df_model.drop(columns=droplist_1)


In [ ]:
target_ghg = "target_GHGEmissionsIntensity(kgCO2e/ft2)"
ghg_model = Linear_reg(dataframe=df_model_ghg, target=target_ghg)


In [ ]:
droplist_2 = [
    "scaled_SourceEUI(kWh/m2)",  # Scaled target
    "target_GHGEmissionsIntensity(kgCO2e/ft2)"  # not to scale
    ]

df_model_eui = df_model.drop(columns=droplist_2)


In [ ]:
eui_target = "target_SourceEUI(kWh/m2)"
eui_model = Linear_reg(dataframe=df_model_eui, target=eui_target)


<hr>

# <u>Tentatives de prediction des données et étude de l'infuence du score Energy Star sur les predictions</u>

## <u>1 : Modeles de regression linéaire sur cible 1 : GHGEmissionsIntensity(kgCO2e/ft2)</u>

### <u>1.1 : Determination hyperparametres et calcul des metriques</u>
### <u>1.2 : Repetition de l'étude 1.1, sur les mêmes données, sans E* Score</u>

<hr>

## <u>2 : Modeles de regression linéire sur cible 2 : SourceEUI(kWh/m2)</u>

### <u>2.1 : Determination hyperparametres et calcul des metriques</u>
### <u>2.2 : Repetition de l'étude 2.1, sur les mêmes données, sans E* Score</u>

<hr>

## <u>3 : Conclusions sur la capacité des modèles prédictifs et sur l'influence du score Energy Star</u>
### <u>3.1 : Remarques sur Energy Star, possible classification</u>

<hr>

## 1 : Regressions lineaires sur l'intensité d'emission de gazs a effets de serre

### <u>1.1 : Determination hyperparametres et calcul des metriques</u>

- La methode utilisée est une approche orientée objet. Cela permet de repeter et affiner les operations plus simplement, et de ne faire l'etape TestTrainSplit qu'une seule fois ; permettant de limiter le caractere aleatoire de cette fonction.
- La methode de validation croisée est, dans un premier temps, la methode par defaut : Leave One Out. Stratified Kfold peut etre utilisé mais limite nos possibilités de visualisation (cf. sklearn.linear_model.RidgeCV)
- Les "candidats" hyperparametres seront : (le lancement des regressions demande un input utilisateur, non visible via jupyter)
    - Ridge : Alpha 0.1 -> 45 , pas de 0.05
    - LASSO : Alpha 0.01 -> 15, pas de 0.01
    - ElasticNet : L1_ratio par defaut (Classe), Alpha 0.1 -> 45 , pas de 0.05


In [ ]:
# Indicated times are impacted by input times -> Necessity to time function executions

ghg_model.execute_all()


In [ ]:
# OLS

ghg_model.std_table


In [ ]:
# Elastic Net

print(ghg_model.elnet_cv.alpha_)
print(ghg_model.elnet_cv.l1_ratio_)
ghg_model.elnet_table


In [ ]:
print(ghg_model.ridge_cv.alpha_)
ghg_model.ridge_table


In [ ]:
ghg_model.ridge_plot()


In [ ]:
print(ghg_model.lasso_best_alpha)
ghg_model.lasso_table


In [ ]:
ghg_model.lasso_plot()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(4, 4),
    dpi=155,
)

ax1 = sns.scatterplot(
    data=ghg_model.df_predictions,
    x="True",
    y="basic_regression"
    )

x = [-1, 1,2,3,4,5]
y = x
ax1.plot(x, y, color="navy", linewidth=.9)
###
# Titles/Lables
ax1.set_ylim(-1, 5)
ax1.set_xlim(-1, 5)
#
###
plt.show()


Comms

### <u>1.2 : Repetition de l'étude 1.1, sur les mêmes données, sans E* Score</u>
- Memes "candidats" hyperparametres :
    - Ridge : Alpha 0.1 -> 45 , pas de 0.05
    - LASSO : Alpha 0.01 -> 15, pas de 0.01
    - ElasticNet : L1_ratio par defaut (Classe), Alpha 0.1 -> 45 , pas de 0.05

In [ ]:
drop_estar = ["ENERGYSTARScore"]
ghg_model.drop_col(col_list=drop_estar)


In [ ]:
ghg_model.execute_all()

In [ ]:
# OLS

ghg_model.std_table


In [ ]:
# Elastic Net

print(ghg_model.elnet_cv.alpha_)
print(ghg_model.elnet_cv.l1_ratio_)
ghg_model.elnet_table


In [ ]:
print(ghg_model.ridge_cv.alpha_)
ghg_model.ridge_table


In [ ]:
ghg_model.ridge_plot()


In [ ]:
print(ghg_model.lasso_best_alpha)
ghg_model.lasso_table


In [ ]:
ghg_model.lasso_plot()


In [ ]:
pplot_data = ghg_model.df_train[["scaled_Electricity(kWh)", "scaled_PropertyArea(SquareMetre)Total",
    "scaled_SourceEUI(kWh/m2)", "scaled_NaturalGas(kWh)", "YearBuilt", "NumberofBuildings",
    "NumberofFloors", "ENERGYSTARScore", "Building_proportion",
    "Parking_proportion", "target_GHGEmissionsIntensity(kgCO2e/ft2)"
    ]]

In [ ]:
ghg_model.df_train.columns

In [ ]:
ghg_model.df_predictions